We run the example (Ishigami) function here - as implemented in SALib - to show it is working as expected.

The expected first order indices (within confidence bounds) are:
* x1: 0.3139
* x2: 0.4424
* x3: 0.0

In [1]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

seed_value = 101
num_samples = 10000

problem = {
  'num_vars': 3,
  'names': ['x1', 'x2', 'x3'],
  'bounds': [[-np.pi, np.pi]]*3
}

param_values = saltelli.sample(problem, num_samples, seed=seed_value)

Y = Ishigami.evaluate(param_values)

Si = sobol.analyze(problem, Y, print_to_console=False)

total, first, second = Si.to_df()

first

,S1,S1_conf
x1,0.310836,0.021076
x2,0.442419,0.016743
x3,-0.003854,0.018390


<hr>

Setting up packages and variables for examples

In [2]:
from SALib.sample import latin, saltelli, sobol_sequence
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import pandas as pd
import numpy as np

num_samples = 1000
seed_value = 101

problem_wo_dummy = {
  'num_vars': 3,
  'names': ['x1', 'x2', 'x3'],
  'bounds': [[-np.pi, np.pi]]*3
}

wo_dummy_values = latin.sample(problem_wo_dummy, num_samples, seed=seed_value)

Y = Ishigami.evaluate(wo_dummy_values)

Si = sobol.analyze(problem_wo_dummy, Y, print_to_console=False)

total, first, second = Si.to_df()

first

,S1,S1_conf
x1,0.124795,0.265309
x2,0.069378,0.203661
x3,0.112679,0.225504


In [3]:
total

,ST,ST_conf
x1,0.976732,0.240514
x2,1.006254,0.350020
x3,1.006333,0.291857


Now we create a wrapping interface that accepts the dummy parameter input (the 4th parameter `x4`), but strips the parameter values from the input array. This has the effect of making the fourth parameter inactive, and therefore insensitive

In [4]:
def ishigami_wrapper(param_values):
    strip_dummy_param = param_values[:, 0:3]
    return Ishigami.evaluate(strip_dummy_param)

SALib problem definition for the above

In [5]:
problem_with_dummy = {
  'num_vars': 4,
  'names': ['x1', 'x2', 'x3', 'x4'],
  'bounds': [[-np.pi, np.pi]]*4
}

You will see below that parameter `x4` comes out as being sensitive even though it should have zero effect

In [6]:
# Generate samples
latin_w_dummy_values = latin.sample(problem_with_dummy, num_samples, seed=seed_value)

# Run model (example)
Y = Ishigami.evaluate(latin_w_dummy_values)

Si = sobol.analyze(problem_with_dummy, Y, print_to_console=False)

total, first, second = Si.to_df()

In [7]:
# Results for 1,000 samples
total

,ST,ST_conf
x1,0.894652,0.221231
x2,0.859920,0.208144
x3,0.857895,0.285130
x4,0.862709,0.230210


In [8]:
first

,S1,S1_conf
x1,0.037732,0.236823
x2,-0.132059,0.303415
x3,-0.074780,0.255443
x4,-0.083835,0.271790


Saltelli sampling does not appear to be affected, but the computational cost to run this analysis - for the purpose of detecting completely insensitive parameters - is comparatively high. 

This can be expected to exponentially increase with the number of parameters

In [9]:
# Generate samples
saltelli_w_dummy_values = saltelli.sample(problem_with_dummy, num_samples, seed=seed_value)

# Run model (example)
Y = Ishigami.evaluate(saltelli_w_dummy_values)

Si = sobol.analyze(problem_with_dummy, Y, print_to_console=False)

total, first, second = Si.to_df()

In [10]:
total

,ST,ST_conf
x1,0.560457,0.095852
x2,0.461343,0.040316
x3,0.247074,0.027137
x4,0.000000,0.000000


In [11]:
first

,S1,S1_conf
x1,0.310486,0.064015
x2,0.453239,0.047442
x3,0.000700,0.055315
x4,0.000000,0.000000


**Runtime of Saltelli sampling with dummy parameters**

In [12]:
%%time
# Generate samples
saltelli_w_dummy_values = saltelli.sample(problem_with_dummy, num_samples, seed=seed_value)

Wall time: 62 ms


In [13]:
%%time
# Run model via the wrapper
Y = ishigami_wrapper(saltelli_w_dummy_values)

Wall time: 18 ms


In [14]:
%%time
Si = sobol.analyze(problem_with_dummy, Y, print_to_console=False)

Wall time: 104 ms


**LHC sampling with dummy parameters**

In [15]:
%%time
# Generate samples
latin_with_dummy_values = latin.sample(problem_with_dummy, num_samples, seed=seed_value)

Wall time: 11 ms


In [16]:
%%time
# Run model directly
Y = Ishigami.evaluate(latin_with_dummy_values)

Wall time: 998 µs


In [17]:
%%time
Si = sobol.analyze(problem_with_dummy, Y, print_to_console=False)

Wall time: 8 ms
